# Mapping cultural remains with object detection
All forests in Sweden, both managed forests and natural old-growth forests in national parks, contain a cultural
heritage. The long history of forest utilization in Sweden has left a rich legacy of diverse types of ancient
monuments and other kinds of cultural remains that document our relationship with the forest and its importance
for Sweden’s development. However, the cultural heritage is too often damaged in forestry operations. The aim of
the project is to do research and develop operationally useful maps that can be used
to identify, protect and enhance the cultural remains in Swedish forests, thereby reducing the destruction of
cultural heritage in our forest landscapes.

In [3]:
!pip install rtree
!pip install torch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


## Topographical modeling

### Select Laz tiles intersecting field data

In [2]:
!python /workspace/code/create_aoi_poolygon.py /workspace/lidar/none.shp /workspace/data/hunting_pits/Fangstgrop_training_Holmen_Cissi_695st_220214.shp /workspace/lidar/pooled_laz_files/ /workspace/data/hunting_pits/laz/ 

/workspace/code/create_aoi_poolygon.py:37: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  main(**args)
Traceback (most recent call last):
  File "/workspace/code/create_aoi_poolygon.py", line 37, in <module>
    main(**args)
  File "/workspace/code/create_aoi_poolygon.py", line 25, in main
    copy_tiles(footprint, field_data, input_directory, output_directory)
  File "/workspace/code/create_aoi_poolygon.py", line 13, in copy_tiles
    intersect = gpd.sjoin(lidar_tiles_footprint, field, how='inner', op='intersects')
  File "/usr/local/lib/python3.8/dist-packages/geopandas/tools/sjoin.py", line 124, in sjoin
    indices = _geom_predicate_query(left_df, right_df, predicate)
  File "/usr/local/lib/python3.8/dist-packages/geopandas/tools/sjoin.py", line 216, in _geom_predicate_query
    sindex = right_df.sindex
  File "/usr/local/lib/python3.8/dist-packages/geopandas/base.py", line 2637, in sindex
   

### Convert selected laz files to DEM

In [ ]:
!python /workspace/code/laz_to_dem.py /workspace/data/hunting_pits/laz/ /workspace/data/hunting_pits/dem_tiles/

### Extract topographical indices

In [ ]:
!python /workspace/code/Extract_topographcical_indices.py /workspace/temp/ /workspace/data/hunting_pits/laz/ /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/ /workspace/data/hunting_pits/topographical_indices_normalized/slope/ /workspace/data/hunting_pits/topographical_indices_normalized/hpmf/ /workspace/data/hunting_pits/topographical_indices_normalized/stdon/

## Labels

**Hunting pits integer masks**

In [ ]:
!python /workspace/code/create_labels.py /workspace/data/hunting_pits/dem_tiles/ /workspace/data/hunting_pits/hunting_pits.shp /workspace/data/hunting_pits/object_detection_data/label_tiles/

All data were split into image chips with the size 256x256. Note that the directories needs to be empty before running the split script. I found it esiest to recreate the directories to avoid errors.

In [132]:
# Start by clearing directories of existing data
import os

#shutil.rmtree('/workspace/data/split_data/') #this fails alot to it manually
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/')
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/labels/')
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/slope/')
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/hillshade/')
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/hpmf/')
os.mkdir('/workspace/data/hunting_pits/object_detection_data/split_data/stdon/')  

# Split data
# Hillshade 
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/ /workspace/data/hunting_pits/object_detection_data/split_data/hillshade/ --tile_size 256
# Slope
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/slope/ /workspace/data/hunting_pits/object_detection_data/split_data/slope/ --tile_size 256
# High pass median filter
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/hpmf/ /workspace/data/hunting_pits/object_detection_data/split_data/hpmf/ --tile_size 256
# High pass median filter
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/stdon/ /workspace/data/hunting_pits/object_detection_data/split_data/stdon/ --tile_size 256
# Labels
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/object_detection_data/label_tiles/ /workspace/data/hunting_pits/object_detection_data/split_data/labels/ --tile_size 256

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 0 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 1
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 316.22img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/18D022_67450_5775_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 400 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 321.47img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/18D022_67475_5750_25.tif are added at /workspace/dat

Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 6000 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 6001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 342.19img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19F047_71050_7225_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 6400 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 6401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 316.94img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19F047_71075_7200_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image

There are 12000 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 12001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 333.07img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19G013_71500_7025_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 12400 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 12401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 307.28img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19G013_71525_7000_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size

There are 18000 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 18001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 345.03img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19G013_71675_7025_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 18400 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 18401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 327.47img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/19G013_71675_7050_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size

There are 24000 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 24001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 343.57img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/20E018_68775_4800_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 24400 files in the /workspace/data/hunting_pits/object_detection_data/split_data/labels/
New image name will start with 24401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 336.45img/s]
400 tiles sample of /workspace/data/hunting_pits/object_detection_data/label_tiles/20E018_68800_4750_25.tif are added at /workspace/data/hunting_pits/object_detection_data/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size

Not all of the splited image chips contained any objects. Chips with less than 1 labeled pixel were removed.

In [133]:
!python /workspace/code/remove_unlabled_chips.py 1 /workspace/data/hunting_pits/object_detection_data/split_data/labels/ /workspace/data/hunting_pits/object_detection_data/split_data/hillshade/ /workspace/data/hunting_pits/object_detection_data/split_data/slope/ /workspace/data/hunting_pits/object_detection_data/split_data/hpmf/ /workspace/data/hunting_pits/object_detection_data/split_data/stdon/

**Hunting pits bounding boxes**\
The segmentation masks were converted to yolo labels for object detection

In [21]:
!pip install pybboxes

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


Convert segmentation masks to YOLO bounding boxes

In [135]:
!python /workspace/code/masks_to_boxes.py /workspace/temp/ /workspace/data/hunting_pits/object_detection_data/split_data/labels/ 256 1 /workspace/data/hunting_pits/object_detection_data/split_data/yolo/ 

# YOLOv5

**partition the dataset into train, validation, and test sets containing 80%, 10%, and 10% of the data, respectively.**

In [165]:
# Read images and annotations
from sklearn.model_selection import train_test_split
images = [os.path.join('/workspace/data/hunting_pits/object_detection_data/split_data/stdon/', x) for x in os.listdir('/workspace/data/hunting_pits/object_detection_data/split_data/stdon/') if x.endswith('.tif')]
annotations = [os.path.join('/workspace/data/hunting_pits/object_detection_data/split_data/yolo/', x) for x in os.listdir('/workspace/data/hunting_pits/object_detection_data/split_data/yolo/') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)
!mkdir /workspace/data/hunting_pits/object_detection_data/images/
!mkdir /workspace/data/hunting_pits/object_detection_data/annotations/
!mkdir /workspace/data/hunting_pits/object_detection_data/images/train /workspace/data/hunting_pits/object_detection_data/images/val /workspace/data/hunting_pits/object_detection_data/images/test /workspace/data/hunting_pits/object_detection_data/annotations/train /workspace/data/hunting_pits/object_detection_data/annotations/val /workspace/data/hunting_pits/object_detection_data/annotations/test

In [114]:
!git clone https://github.com/ivder/YoloBBoxChecker.git

Cloning into 'YoloBBoxChecker'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), 5.94 KiB | 74.00 KiB/s, done.


In [115]:
!ls

 Dockerfile
 Extract_topographcical_indices.py
 LICENSE
'Laz to DEM.ipynb'
'Mapping cultural remains with object detection.ipynb'
 README.md
 Select_chips_with_labels.py
 Select_study_areas.py
 Untitled1.ipynb
 Untitled2.ipynb
'Williams notes.ipynb'
 YoloBBoxChecker
 __pycache__
 create_aoi_poolygon.py
 create_labels.py
 data
 evaluate_model.py
 images
 inference.py
 inspect_distribution.py
 laz_to_dem.py
 lidar_tile_footprint.py
 masks_to_boxes.py
 post_processing.py
 prepare_the_moon.py
 remove_unlabled_chips.py
 select_laz_tiles.py
 select_lidar_tiles.py
 split_training_data.py
 train.py
 utils


In [151]:
!python /workspace/code/YoloBBoxChecker/main.py

Input:/workspace/data/hunting_pits/object_detection_data/8bitimage/0022.txt
Output:/workspace/data/hunting_pits/object_detection_data/results/0022.tif
